# Debugging A Simple Image Processing Program Using The Compute Sanitizer
## 05 Racecheck

In this section, we will introduce the racecheck tool. Racecheck can detect and
report possible data race on shared memory. Such data-race can occur when there
is at least a write access and a read or write without any form of
synchronization (thread barriers, warp synchronization, etc.) in between, on different threads to the same (shared) memory
address. Data races can be the source of unexpected and invalid results, sometimes intermittently.

Our application seems to have a correct output, but data races may only manifest themselves intermittently on unpredictable factors. Since our application does rely on shared memory, it can't hurt to verify that we don't have such data race:

```sh
    compute-sanitizer --tool=racecheck ./convolution
```

We can observe that we do have errors that are reported.

```
========= COMPUTE-SANITIZER
Image ./checkerboard.png size: 32 * 32
Image ./icon.png size: 32 * 32
Grid size: 2 * 2 * 1
Block size: 31 * 31 * 1
========= ERROR: Race reported between Read access at 0x6360 in /home/nvidia/sanitizer-demo/convolution.cu:123:convolution(pixel const*, int, int, float const*, int, pixel*)
=========     and Write access at 0x1fe0 in /home/nvidia/sanitizer-demo/convolution.cu:63:convolution(pixel const*, int, int, float const*, int, pixel*) [1088 hazards]
=========     and Write access at 0x26d0 in /home/nvidia/sanitizer-demo/convolution.cu:68:convolution(pixel const*, int, int, float const*, int, pixel*) [64 hazards]
=========     and Write access at 0x3590 in /home/nvidia/sanitizer-demo/convolution.cu:79:convolution(pixel const*, int, int, float const*, int, pixel*) [64 hazards]
=========     and Write access at 0x4480 in /home/nvidia/sanitizer-demo/convolution.cu:89:convolution(pixel const*, int, int, float const*, int, pixel*) [32 hazards]
=========     and Write access at 0x5380 in /home/nvidia/sanitizer-demo/convolution.cu:99:convolution(pixel const*, int, int, float const*, int, pixel*) [32 hazards]
========= 
[...]
```

There appear to be indeed a data race even though it may not have visible consequences in our context. But it may fail randomly when we don't expect it, so we should have it corrected. Take a moment to examine the reported errors and try to find the mistake. Unfold the next cell to reveal the solution:

The mistake is that we lack synchronization between the phase where we copy our
data from global to shared memory, and the phase where we use it. Adding a thread barrier with
`__syncthreads()` at the line 108 will fix the error, that we can verify by
recompiling and re-running racecheck.

We should now have the expected output for [icon-out.png](code/icon-out.png) and [checkerboard-out.png](code/checkerboard-out.png) with certainty.

[Go to step 06](06_initcheck.ipynb)